[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EMQA_garch_oil/EMQA_garch_oil.ipynb)

# EMQA_garch_oil
GARCH(1,1) estimation on Brent crude oil returns.
**Output:** `garch_oil.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import yfinance as yf

def fetch(ticker, start='2020-01-01', end='2025-12-31'):
    d = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(d.columns, pd.MultiIndex):
        return d['Close'].squeeze().dropna()
    return d['Close'].dropna()


In [ ]:
# Fetch Brent returns
brent = fetch('BZ=F', start='2018-01-01')
log_ret = (np.log(brent / brent.shift(1)).dropna()) * 100  # percentage returns
print(f"Returns: {len(log_ret)} obs")

In [ ]:
# Fit GARCH(1,1)
try:
    from arch import arch_model
    am = arch_model(log_ret, vol='Garch', p=1, q=1, dist='normal', mean='Constant')
    res = am.fit(disp='off')
    print(res.summary())
    cond_vol = res.conditional_volatility
    use_arch = True
except ImportError:
    print("arch package not available -- simulating GARCH(1,1)")
    use_arch = False
    omega, alpha, beta_ = 0.01, 0.08, 0.90
    n = len(log_ret)
    cond_var = np.zeros(n)
    cond_var[0] = log_ret.var()
    ret_vals = log_ret.values
    for t in range(1, n):
        cond_var[t] = omega + alpha * ret_vals[t-1]**2 + beta_ * cond_var[t-1]
    cond_vol = pd.Series(np.sqrt(cond_var), index=log_ret.index)

In [ ]:
# 2-panel plot
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Panel 1 - Returns with +/- 2 sigma bands
axes[0].plot(log_ret.index, log_ret.values, color=COLORS['gray'], linewidth=0.4, alpha=0.7, label='Returns')
axes[0].plot(cond_vol.index, 2 * cond_vol.values, color=COLORS['red'], linewidth=1, label='+2$\\sigma$')
axes[0].plot(cond_vol.index, -2 * cond_vol.values, color=COLORS['red'], linewidth=1, label='-2$\\sigma$')
axes[0].set_ylabel('Returns (%)')
axes[0].set_title('Brent Crude - GARCH(1,1) Conditional Volatility Bands')
axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), frameon=False, ncol=3)

# Panel 2 - Conditional volatility
axes[1].plot(cond_vol.index, cond_vol.values, color=COLORS['blue'], linewidth=1)
axes[1].set_ylabel('Conditional Volatility (%)')
axes[1].set_xlabel('Date')
axes[1].set_title('GARCH(1,1) Conditional Volatility')

plt.tight_layout()
save_fig(fig, 'garch_oil.pdf')
plt.show()